In [211]:
import requests
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup as bs
import sys

def MyBookie ():
    URL = 'https://www.mybookie.ag/sportsbook/mlb/'

    content = requests.get(URL).text

    data = bs(content,"html.parser")

    #Entire betting info for all bets on this specific sport (For MLB there are 3 genres of bets: World Series Winner, Live Betting, Future Game Betting)
    SportInfo = data.find_all("div", class_='line-default')


    GameBets = []

    for counter in range (0, len(SportInfo)):
        GameInfo=SportInfo[counter].find_all("div", class_ = 'game-line py-3')
        

        #Specific Game
        for increaser in range (1, len(GameInfo)):

            TimeInfo = GameInfo[increaser].meta['content'] #Expanded start time as string 
            GameTime = datetime.strptime(TimeInfo[0:(len(TimeInfo)-6)], '%Y-%m-%d %H:%M') #Time in datetime form, allows easy comparison to other betting sites 

            


            VisitorTeamName = GameInfo[increaser].find_all("a")[0].text #Visitor Team Name
            HomeTeamName = GameInfo[increaser].find_all("a")[1].text #Home Team Name

            #print(VisitorTeamName, " at ", HomeTeamName)

            
            HomeInfo=GameInfo[increaser].find("div", class_ = 'game-line__home-line') #All the ways to bet on home team (spread, moneyline, total)
            VisitorInfo=GameInfo[increaser].find("div", class_ = 'game-line__visitor-line') #All the ways to bet on visitor team (spread, moneyline, total)

            HomeBetInfo = HomeInfo.find_all("button") #HomeInfo broken up into 3 parts, spread, moneyline, total
            VisitorBetInfo = VisitorInfo.find_all("button") #VisitorInfo broken up into 3 parts, spread, moneyline, total

            

            for pluser in range (0, len(HomeBetInfo)): #Getting all odds for each team for each way to bet (spread, moneyline, total)
                
                idPartOne = (TimeInfo.replace(" ",""))
                idPartTwo = (idPartOne[0:(len(idPartOne)-6)].replace("-",""))

                GameID = HomeTeamName.split()[len(HomeTeamName.split())-1]+VisitorTeamName.split()[(len(VisitorTeamName.split())-1)]+idPartTwo #GameID format: Home Team Name + Visitor Team Name + year + month + day + hour + minute 
            
                
                if HomeBetInfo[pluser]['data-wager-type'] == 'sp':
                    HomeSpread = HomeBetInfo[pluser]['data-spread']
                    HomeSpreadOdds = HomeBetInfo[pluser]['data-odds']
                    HomeSpreadBetID = GameID+"|H_sp||"+HomeSpread+"|"+HomeSpreadOdds #BetID format: GameID + Home/Away bet type, + Spread (if applicable) + odds

                    VisitorSpread = VisitorBetInfo[pluser]['data-spread']
                    VisitorSpreadOdds = VisitorBetInfo[pluser]['data-odds']
                    VisitorSpreadBetID = GameID+"|V_sp||"+VisitorSpread+"|"+VisitorSpreadOdds
                
                if HomeBetInfo[pluser]['data-wager-type'] == 'ml':
                    HomeMLOdds = HomeBetInfo[pluser]['data-odds']
                    HomeMLOddsBetID = GameID+"|H_ml||"+HomeMLOdds

                    VisitorMLOdds = VisitorBetInfo[pluser]['data-odds']
                    VisitorMLOddsBetID = GameID+"|V_ml||"+VisitorMLOdds

                if HomeBetInfo[pluser]['data-wager-type'] == 'to':
                    HomeTotal = HomeBetInfo[pluser]['data-spread']
                    HomeTotalOdds = HomeBetInfo[pluser]['data-odds']
                    HomeTotalBetID = GameID+"|H_to||"+HomeTotal+"|"+HomeTotalOdds

                    VisitorTotal = VisitorBetInfo[pluser]['data-spread']
                    VisitorTotalOdds = VisitorBetInfo[pluser]['data-odds']
                    VisitorTotalBetID = GameID+"|V_to||"+VisitorTotal+"|"+VisitorTotalOdds


        

            HomeSpreadList = [HomeTeamName, VisitorTeamName, GameTime, HomeSpread, HomeSpreadOdds, GameID, HomeSpreadBetID]
            VisitorSpreadList = [VisitorTeamName, HomeTeamName, GameTime, VisitorSpread, VisitorSpreadOdds, GameID, VisitorSpreadBetID]

            #print(HomeSpreadList)
            #print(VisitorSpreadList)

            HomeMlList = [HomeTeamName, VisitorTeamName, GameTime, HomeMLOdds, GameID, HomeMLOddsBetID]
            VisitorMlList = [VisitorTeamName, HomeTeamName, GameTime, VisitorMLOdds, GameID, VisitorMLOddsBetID]

            #print(HomeMlList)
            #print(VisitorMlList)

            HomeTotalList = [HomeTeamName, VisitorTeamName, GameTime, HomeTotal, HomeTotalOdds, GameID, HomeTotalBetID]
            VisitorTotalLst = [VisitorTeamName, HomeTeamName, GameTime, VisitorTotal, VisitorTotalOdds, GameID, VisitorTotalBetID]

            #print(HomeTotalList)
            #print(VisitorTotalLst)


        

            AllGameBets = [HomeSpreadBetID, VisitorSpreadBetID, HomeMLOddsBetID, VisitorMLOddsBetID, HomeTotalBetID, VisitorTotalBetID]

            GameBets.append(AllGameBets)

    #print(GameBets)

    #for counter in range(0,len(GameBets)):
        #for increaser in range (0,len(GameBets[counter])):
            #print(GameBets[counter][increaser])
       # print("\n")
            

    return GameBets
    
def DraftKings ():

    URL = 'https://sportsbook.draftkings.com/leagues/baseball/88670847?wpsrc=Organic%20Search&wpaffn=Google&wpkw=https%3A%2F%2Fsportsbook.draftkings.com%2Fleagues%2Fbaseball%2F88670847&wpcn=leagues&wpscn=baseball%2F88670847'



    content = requests.get(URL).text

    data = bs(content,"lxml")

    SportInfo = data.find_all("div", class_='parlay-card-10-a') #Length is 3

    AllBetInfo = []

    for n in range (0, len(SportInfo)):
        Body = ((SportInfo[n].find(class_="sportsbook-table__body"))) #Body of games today

        GameInfo = Body.find_all("tr") # each games individual data

        #print(GameInfo[0])

        for counter in range (0, len(GameInfo)):  
        # print(GameInfo[counter].find("div", class_="event-cell__name-text").text) #Team Name

            if (counter+1) % 2 == 0:
                #print("vs ",GameInfo[counter-1].find("div", class_="event-cell__name-text").text)

                HomeTeamName = GameInfo[counter].find("div", class_="event-cell__name-text").text
                VisitorTeamName = GameInfo[counter-1].find("div", class_="event-cell__name-text").text
                TypePartOne = 'H'
            else:
                #print("vs ",GameInfo[counter+1].find("div", class_="event-cell__name-text").text)

                HomeTeamName = GameInfo[counter+1].find("div", class_="event-cell__name-text").text
                VisitorTeamName = GameInfo[counter].find("div", class_="event-cell__name-text").text
                TypePartOne = 'V'
                
            #print(SportInfo[1].tr.span.text) #Date of game (Will have to use counter the same way you will have to with describing Body)
            
            if SportInfo[n].tr.span.text == 'Today ': #Gets numerical date of today
                Date = str(date.today()).replace("-","")
                #print(Date)


            if SportInfo[n].tr.span.text == 'Tomorrow ': #Gets numerical date of tomorrow 
                Date = str(date.today()+timedelta(1)).replace("-","")
                #print(Date)

            if len(GameInfo[counter].span.text) == 0: #Determine whether or not the game is live
                #print("Live")
                timeFormat = 'LIVE'

            else: #Getting time in proper format
                time = GameInfo[counter].span.text[0: (len(GameInfo[counter].span.text)-2)] #Start time in AM/PM format without adjusting for timezone

                #Adjusting for format and timezone
                if int(time[0:time.find(":")])-4 < 0:
                    if GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-2):len(GameInfo[counter].span.text)] == 'AM':
                        timeFormat = str((int(time[0:time.find(":")])-4)+24) + GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-5):(len(GameInfo[counter].span.text)-2)]
                    else:
                        timeFormat = str((int(time[0:time.find(":")])-4)+24) + GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-5):(len(GameInfo[counter].span.text)-2)]
                        
                else:
                    if GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-2):len(GameInfo[counter].span.text)] == 'AM':
                        timeFormat = str((int(time[0:time.find(":")])-4)+12) + GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-5):(len(GameInfo[counter].span.text)-2)]
                    else:
                        if len(str(int(time[0:time.find(":")])-4+12)) == 1: #Will put a 0 in front of times like 9:30 to make it 09:30
                            timeFormat = "0" + str((int(time[0:time.find(":")])-4)+12) + GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-5):(len(GameInfo[counter].span.text)-2)]
                        else:
                            timeFormat = str((int(time[0:time.find(":")])-4)+12) + GameInfo[counter].span.text[(len(GameInfo[counter].span.text)-5):(len(GameInfo[counter].span.text)-2)]

                #timeFormat is the correctly formatted time that can e used for the entirety of the game
            #print(timeFormat)
            #print(len(GameInfo[counter].find_all("td", class_="sportsbook-table__column-row"))) #Three sections all titled sportsbook-table__column-row that respectively have info on spread, total and moneyline

            

            BetInfo = GameInfo[counter].find_all("td", class_="sportsbook-table__column-row")
            for increaser in range (0, len(BetInfo)):
                #print(BetInfo[increaser].find("span", class_= "sportsbook-outcome-cell__label")) #Prints O or U if applicable 
                tempOU = BetInfo[increaser].find("span", class_= "sportsbook-outcome-cell__label")

                if tempOU != None:
                    TypePartTwo = TypePartOne + "_to"

                #print(BetInfo[increaser].find("span", class_="sportsbook-outcome-cell__line")) #Prints spread/total if applicable
                tempSpread = BetInfo[increaser].find("span", class_="sportsbook-outcome-cell__line")

                if tempSpread != None:
                    if tempOU != None:
                        if tempOU.text == 'U':
                            tempSpread = "-"+ tempSpread.text
                            if tempSpread[0] == '+':
                                tempSpread = tempSpread[1:len(tempSpread)] #Removes plus symbol to align with other sportsbooks
                            #print(tempSpread)
                        else:
                            tempSpread = tempSpread.text
                            if tempSpread[0] == '+':
                                tempSpread = tempSpread[1:len(tempSpread)] #Removes plus symbol to align with other sportsbooks
                            #print(tempSpread)
                    else:
                        tempSpread = tempSpread.text
                        if tempSpread[0] == '+':
                                tempSpread = tempSpread[1:len(tempSpread)] #Removes plus symbol to align with other sportsbooks
                        #print(tempSpread)
                        TypePartTwo = TypePartOne + "_sp"
                

                #print(BetInfo[increaser].find("span", class_= "sportsbook-odds american default-color")) #Prints the odds of spread and total
                tempSpreadTotalOdds = BetInfo[increaser].find("span", class_= "sportsbook-odds american default-color")
                
                if tempSpreadTotalOdds != None:
                    #print(tempSpreadTotalOdds.text) 
                    tempSpreadTotalOdds = tempSpreadTotalOdds.text 
                    if tempSpreadTotalOdds[0] == '+':
                                tempSpreadTotalOdds = tempSpreadTotalOdds[1:len(tempSpreadTotalOdds)]  #Removes plus symbol to align with other sportsbooks

                #print(BetInfo[increaser].find("span", class_= "sportsbook-odds american no-margin default-color")) #Prints the odds of moneyline 
                tempMLOdds = BetInfo[increaser].find("span", class_= "sportsbook-odds american no-margin default-color")
                #print(tempMLOdds)
                if tempMLOdds != None:
                    #print(tempMLOdds.text)
                    tempMLOdds = tempMLOdds.text
                    TypePartTwo = TypePartOne + "_ml"
                    if tempMLOdds[0] == '+':
                                tempMLOdds = tempMLOdds[1:len(tempMLOdds)]  #Removes plus symbol to align with other sportsbooks
                    #print(tempMLOdds)
                    #print(HomeTeamName.split()[len(HomeTeamName.split())-1]+VisitorTeamName.split()[(len(VisitorTeamName.split())-1)]+Date+timeFormat+"|"+TypePartTwo+"|"+tempMLOdds)
                    IndividualBetInfo = HomeTeamName.split()[len(HomeTeamName.split())-1]+VisitorTeamName.split()[(len(VisitorTeamName.split())-1)]+Date+timeFormat+"|"+TypePartTwo+"||"+tempMLOdds


                if tempSpreadTotalOdds != None:
                    IndividualBetInfo = HomeTeamName.split()[len(HomeTeamName.split())-1]+VisitorTeamName.split()[(len(VisitorTeamName.split())-1)]+Date+timeFormat+"|"+TypePartTwo+"|"+tempSpread+"||"+tempSpreadTotalOdds
                    #print(HomeTeamName.split()[len(HomeTeamName.split())-1]+VisitorTeamName.split()[(len(VisitorTeamName.split())-1)]+Date+timeFormat+"|"+TypePartTwo+"|"+tempSpread+"|"+tempSpreadTotalOdds)

                if tempSpreadTotalOdds == None:
                    if tempMLOdds == None:
                    #print("Incomplete Bet Slip")
                        continue

                AllBetInfo.append(IndividualBetInfo)


    #print(AllBetInfo)

    #increaser = 1
    #for counter in range (0, len(AllBetInfo)):
        #print(AllBetInfo[counter])
        #if increaser == 6:
            #print("\n")
            #increaser = 1
            #continue
        #increaser+=1
    
    return AllBetInfo

#print("MY BOOKIE")
MyBookieOdds = MyBookie()
#print(MyBookieOdds[0])


#print("DRAFTKINGS")
DraftKingsOdds = DraftKings()

print("Bet matches between DraftKings and MyBookie")

for practiceCounter in range (0, len(DraftKingsOdds)):

    #print(DraftKingsOdds[practiceCounter])


    if DraftKingsOdds[practiceCounter].find("V_") != -1: #Bet is for visitor team 


        #if spread/total
        if (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+5)]) != '|': #Bet is spread/total


            if (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+5)]) == '-': #Spread/total is negative and coresponding one needs to be positive
                #print (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+6):(DraftKingsOdds[practiceCounter].find("||"))])
                #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+5] + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+6):(DraftKingsOdds[practiceCounter].find("||"))]) 
                OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+5] + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+6):(DraftKingsOdds[practiceCounter].find("||"))]

            else:  #Spread/total is positive and coresponding one needs to be negative 
                #print ("-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+5):(DraftKingsOdds[practiceCounter].find("||"))])
                #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+5]+ "-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+5):(DraftKingsOdds[practiceCounter].find("||"))]) 
                OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+5]+ "-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+5):(DraftKingsOdds[practiceCounter].find("||"))]

        else:  #Bet is moneyline
            #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+4])
            OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("V_")] + "H" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("V_")+1):(DraftKingsOdds[practiceCounter].find("V_"))+4]

    else: #Bet is for home team

            #if spread/total
        if (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+5)]) != '|': #Bet is spread/total
            

            if (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+5)]) == '-': #Spread/total is negative and coresponding one needs to be positive
                #print (DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+6):(DraftKingsOdds[practiceCounter].find("||"))])
                #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+5] + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+6):(DraftKingsOdds[practiceCounter].find("||"))]) 
                OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+5] + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+6):(DraftKingsOdds[practiceCounter].find("||"))]

            else:  #Spread/total is positive and coresponding one needs to be negative 
                #print ("-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+5):(DraftKingsOdds[practiceCounter].find("||"))])
                #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+5]+ "-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+5):(DraftKingsOdds[practiceCounter].find("||"))])
                OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+5]+ "-" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+5):(DraftKingsOdds[practiceCounter].find("||"))]

        else:  #Bet is moneyline
            #print(DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+4])
            OppID = DraftKingsOdds[practiceCounter][0:DraftKingsOdds[practiceCounter].find("H_")] + "V" + DraftKingsOdds[practiceCounter][(DraftKingsOdds[practiceCounter].find("H_")+1):(DraftKingsOdds[practiceCounter].find("H_"))+4]

    #print(OppID)

    for counter in range (0,len(MyBookieOdds)): #Finds bet matches
        for increaser in range (0, len(MyBookieOdds[counter])):
            if OppID in str(MyBookieOdds[counter][increaser]):

                #print("My Bookie")
                #print(str(MyBookieOdds[counter][increaser]))
                #print(MyBookieOdds[counter][increaser][MyBookieOdds[counter][increaser].find("||")+2:len(MyBookieOdds[counter][increaser])])
                MyBookieBetOdds = int(MyBookieOdds[counter][increaser][MyBookieOdds[counter][increaser].find("||")+2:len(MyBookieOdds[counter][increaser])])

                if MyBookieBetOdds < 0:
                    IPMyBookieBetOdds = (-1*MyBookieBetOdds) / ((-1*MyBookieBetOdds)+100)
                else:
                    IPMyBookieBetOdds = 100 / (MyBookieBetOdds+100)

                #print("DraftKings")
                #print(DraftKingsOdds[practiceCounter])
                #print(DraftKingsOdds[practiceCounter][DraftKingsOdds[practiceCounter].find("||")+2: len(DraftKingsOdds[practiceCounter])])
                DraftKingsBetOdds = int(DraftKingsOdds[practiceCounter][DraftKingsOdds[practiceCounter].find("||")+2: len(DraftKingsOdds[practiceCounter])])

                if DraftKingsBetOdds < 0:
                    IPDraftKingsBetOdds = (-1*DraftKingsBetOdds) / ((-1*DraftKingsBetOdds)+100)
                else:
                    IPDraftKingsBetOdds = 100 / (DraftKingsBetOdds+100)


                if IPDraftKingsBetOdds + IPMyBookieBetOdds < 1:
                    print("OPPORTUNITY: Expected Win=", ((1-IPDraftKingsBetOdds + IPMyBookieBetOdds)*100), "%")
                    #print(IPDraftKingsBetOdds + IPMyBookieBetOdds)
                else:
                    print("No arbitrage. Expected loss =", ((1-(IPDraftKingsBetOdds + IPMyBookieBetOdds))*100), "%")
                    #print(IPDraftKingsBetOdds + IPMyBookieBetOdds)
                
                break
        else:
            continue
        break



Bet matches between DraftKings and MyBookie
No arbitrage. Expected loss = -2.553191489361706 %
No arbitrage. Expected loss = -5.0167224080267525 %
No arbitrage. Expected loss = -4.545454545454541 %
No arbitrage. Expected loss = -4.545454545454541 %
No arbitrage. Expected loss = -5.8693244739756345 %
No arbitrage. Expected loss = -3.600464576074325 %
